In [95]:
import numpy as np
import pandas as pd
import nltk
import os
from bs4 import BeautifulSoup
import pdfplumber
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
import re

In [25]:
#Download necessary nltk packages
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sgray\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sgray\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [101]:
#Function for rippin words from pdf
def extract_text_from_pdf(data_dir, pdf_name, folder = None):
    """Extracts text from a PDF file.
            data_dir: Data directory
            pdf_name: Name of pdf file
            folder (optional): Specific folder in data directory to pull from"""
    if folder != None:
        pdf_folder = os.path.join(data_dir, folder)
        pdf_file = os.path.join(pdf_folder, pdf_name)
    else:
        pdf_file = os.path.join(data_dir, pdf_name)
    with pdfplumber.open(pdf_file) as pdf:
        text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
        return(text)

#Function for summarizin text
def summarize_text(text, num_sentences=5):
    """Summarizes the text using sentence ranking based on word frequency."""
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    
    # Tokenize words and filter out stopwords
    words = word_tokenize(text.lower())
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word.isalnum() and word not in stop_words]

    # Compute word frequency
    word_freq = Counter(words)

    # Score sentences based on word frequency
    sentence_scores = {sent: sum(word_freq.get(word, 0) for word in word_tokenize(sent.lower())) for sent in sentences}

    # Select the top 'num_sentences' sentences
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    
    return " ".join(summary_sentences)

#TextRank summarizer, better than normal
def summarize_text_textrank(text, num_sentences=3):
    """Summarizes text using the TextRank algorithm."""
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return " ".join(str(sentence) for sentence in summary)

#Split by headers
def split_by_headers(text):
    """split text into sections based on headers"""
    sections = {}
    current_header = "Introduction"
    sections[current_header] = []

    #detect headers through newline or all caps
    header_pattern = re.compile(r'(^[A-Z][A-Z\s\d]+$)', re.MULTILINE)

    lines = text.split("\n")
    for line in lines:
        if header_pattern.match(line.strip()): #compare to current line
            current_header = line.strip()
            sections[current_header] = []
        else:
            sections[current_header].append(line.strip())

    return({header: "\n".join(content) for header, content in sections.items() if content})

#New Summarizer for sections
def summarize_section(text, num_sentences=3):
    """Summarizes text using the TextRank algorithm."""
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return " ".join(str(sentence) for sentence in summary)

In [111]:
#Data directory
data_dir = r'C:\Users\sgray\OneDrive - University of Tennessee\Spring 25\DATA Capstone\docs'#\good_fit'
#Folder name (optional), will ignore if you don't have subfolder to look through
folder = "good_fit"
#grab pdf_name
pdf_name = "RFP+CR-346570+Attachment+A+SOW+SQA+Consulting.pdf"
#Append pdf_name to end of Data directory, add folder if necessary
#Then grab the text from it
text = extract_text_from_pdf(data_dir, pdf_name, folder)
#Split by headers 
"""WIP: NOT WORKING CORRECTLY"""
sections = split_by_headers(text)
#Spit out summaries under each header
summaries = {header: summarize_section(content, num_sentences = 3) for header, content in sections.items()}

In [113]:
for header, summary in summaries.items():
    print(f"==={header}===")
    print(summary)
    print('\n')

===Introduction===
Attachment A Statement of Work Software Quality Assurance (SQA) Consultant August 23, 2023 Overview/Objective: Vendor to provide SQA consulting service to Fermi National Accelerator Laboratory. Requirements: The vendor must be experienced in SQA consulting within the Department of Energy (DOE) and working with DOE National Laboratories on Responsibilities/Deliverables: The vendor will be asked to provide the following: • A thorough review of FRA SQA Program documentation • Create an assessment plan for the SQA Review (lab-wide) • Execute the SQA Review • Support the development of improvement actions.


